# Neural network prediction for the sPOD-DL-ROM for CombustionWave

In [ ]:
import sys
sys.path.append('../DL/')
sys.path.append('../sPOD/lib/')
sys.path.append('../DL-ROM/LIB/')

In [ ]:
from synthetic import synthetic
import numpy as np

## Data generation / Shifted POD of the data

In [ ]:
df = synthetic(spod_iter=200)        

## Input data for the network 

In [ ]:
# We collect the time amplitudes, shifts and the parameters for the training as well as the testing data.
TA_TRAIN = df.TA_TRAIN
SHIFTS_TRAIN = df.SHIFTS_TRAIN
PARAMS_TRAIN = df.PARAMS_TRAIN
TA_TEST = df.TA_TEST
SHIFTS_TEST = df.SHIFTS_TEST
PARAMS_TEST = df.PARAMS_TEST
TA_POD_TRAIN = df.TA_POD_TRAIN
TA_POD_TEST = df.TA_POD_TEST

In [ ]:
# Save the values for future use 

In [ ]:
print("Grid, Nx : {}, Nt : {}".format(df.Nx, df.Nt))
print("Number of sPOD frames : {}".format(df.NumFrames))
print("Number of modes per frame : {}".format(df.nmodes))
print("Number of parameter instances : {}".format(int(int(TA_TRAIN.shape[1]) / df.Nt)))
print("Size of training matrix : {} x {}".format(int(TA_TRAIN.shape[0]), int(TA_TRAIN.shape[1])))

In [ ]:
# This cell is reserved for any data manipulations that need to be done for the network input
shifts_train = np.concatenate((np.reshape(SHIFTS_TRAIN[0], newshape=[1, -1]), np.reshape(SHIFTS_TRAIN[1], newshape=[1, -1])), axis=0)
shifts_test = np.concatenate((np.reshape(SHIFTS_TEST[0], newshape=[1, -1]), np.reshape(SHIFTS_TEST[1], newshape=[1, -1])), axis=0)

ta_train = np.concatenate((TA_TRAIN, shifts_train), axis=0)
ta_test = np.concatenate((TA_TEST, shifts_test), axis=0)

## Network call

## Feed forward neural network

In [ ]:
# Path for the pretrained weights
PATH_sPOD = 'DNN_result/syntheticOnlyTA/training_results_sPOD/2022_08_25__12-34-47/trained_weights/weights.pt'
PATH_POD = 'DNN_result/synthetic/training_results_sPOD/2022_08_23__18-56-21/trained_weights/weights.pt'

In [ ]:
params_sPOD = {
        'scaling': True,  # true if the data should be scaled
        'full_order_model_dimension': df.Nx,  # N_h
        'reduced_order_model_dimension': df.nmodes * df.NumFrames + df.NumFrames,  # N
        'totalModes': df.nmodes * df.NumFrames,  # Total number of modes for all the frames
        'NumOfFrames': df.NumFrames  # Total number of frames
    }
params_POD = {
        'scaling': True,  # true if the data should be scaled
        'full_order_model_dimension': df.Nx,  # N_h
        'reduced_order_model_dimension': df.nmodes * df.NumFrames + df.NumFrames,  # N
        'totalModes': df.nmodes * df.NumFrames,  # Total number of modes for all the frames
        'NumOfFrames': 0  # Total number of frames
    }

In [ ]:
# training the model
from network import run_model 
print("#################################")
print("sPOD-DL-ROM")
trained_model_sPOD, scaling_sPOD = run_model(ta_train, PARAMS_TRAIN, epochs=1000, lr=0.05, loss='L1', 
                       logs_folder='./DNN_result/synthetic/training_results_sPOD',
                      pretrained_load=False, pretrained_weights=PATH_sPOD, params=params_sPOD)
print("#################################\n")
print("#################################")
print("POD-DL-ROM")
trained_model_POD, scaling_POD = run_model(TA_POD_TRAIN, PARAMS_TRAIN, epochs=1000, lr=0.05, loss='L1', 
                      logs_folder='./DNN_result/synthetic/training_results_POD',
                     pretrained_load=False, pretrained_weights=PATH_POD, params=params_POD)
print("#################################\n")

In [ ]:
# loading the model
import torch
import pathlib
import os

log_folder_base_sPOD = 'DNN_result/synthetic/training_results_sPOD/'
log_folder_trained_model_sPOD = sorted(pathlib.Path(log_folder_base_sPOD).glob('*/'), key=os.path.getmtime)[-1]
PATH_sPOD = str(log_folder_trained_model_sPOD) + '/trained_weights/' + 'weights.pt'


log_folder_base_POD = 'DNN_result/synthetic/training_results_POD/'
log_folder_trained_model_POD = sorted(pathlib.Path(log_folder_base_POD).glob('*/'), key=os.path.getmtime)[-1]
PATH_POD = str(log_folder_trained_model_POD) + '/trained_weights/' + 'weights.pt'


In [ ]:
from network import scale_params
PARAMS_TEST_sPOD = scale_params(PARAMS_TEST, params_sPOD, scaling_sPOD)
PARAMS_TEST_POD = scale_params(PARAMS_TEST, params_POD, scaling_POD)

In [ ]:
# testing the model
from network import test_model 
rel_err_sPOD, results_predicted_sPOD = test_model(ta_test, PARAMS_TEST_sPOD, 
                                                  saved_model=True, 
                                                  PATH_TO_WEIGHTS=PATH_sPOD, params=params_sPOD,
                                                  scaling=scaling_sPOD) 
rel_err_POD, results_predicted_POD = test_model(TA_POD_TEST, PARAMS_TEST_POD, 
                                                saved_model=True,
                                               PATH_TO_WEIGHTS=PATH_POD, params=params_POD,
                                               scaling=scaling_POD)
print(rel_err_sPOD, rel_err_POD)

## Convolutional autoencoder and DNN coupled model

In [ ]:
from TrainingFramework import TrainingFramework
from TestingFramework import TestingFramework
import Helper

In [ ]:
dict_network_sPOD = {
        'time_amplitude_train': ta_train,
        'time_amplitude_test': ta_test,
        'parameter_train': PARAMS_TRAIN,
        'parameter_test': PARAMS_TEST,
        'batch_size': 500,
        'num_early_stop': 500,  # Number of epochs for the early stopping
        'pretrained_load': False,  # Wthere to initialize the network with pretrained weights
        'scaling': True,  # true if the data should be scaled
        'perform_svd': 'randomized',  # 'normal', 'randomized'
        'learning_rate': 0.0005,  # eta
        'full_order_model_dimension': df.Nx,  # N_h
        'reduced_order_model_dimension': df.nmodes * df.NumFrames + df.NumFrames,  # N
        'encoded_dimension': 4,  # dimension of the system after the encoder
        'omega_h': 0.8,
        'omega_N': 0.2,
        'typeConv': '1D',  # Type of convolutional layer for the network : '1D' or '2D'
        'totalModes': df.nmodes * df.NumFrames,  # Total number of modes for all the frames
        'NumOfFrames': df.NumFrames  # Total number of frames
    }


dict_network_POD = {
        'time_amplitude_train': TA_POD_TRAIN,
        'time_amplitude_test':TA_POD_TEST,
        'parameter_train': PARAMS_TRAIN,
        'parameter_test': PARAMS_TEST,
        'batch_size': 500,
        'num_early_stop': 10000,  # Number of epochs for the early stopping
        'pretrained_load': False,  # Wthere to initialize the network with pretrained weights
        'scaling': True,  # true if the data should be scaled
        'perform_svd': 'randomized',  # 'normal', 'randomized'
        'learning_rate': 0.05,  # eta
        'full_order_model_dimension': df.Nx,  # N_h
        'reduced_order_model_dimension': df.nmodes * df.NumFrames + df.NumFrames,  # N
        'encoded_dimension': 4,  # dimension of the system after the encoder
        'omega_h': 0.8,
        'omega_N': 0.2,
        'typeConv': '1D',  # Type of convolutional layer for the network : '1D' or '2D'
        'totalModes': df.nmodes * df.NumFrames + df.NumFrames,  # Total number of modes for all the frames
        'NumOfFrames': 0  # Total number of frames
    }


## -------------------------------------

In [ ]:
# select the path to the pre trained weights
PATH_sPOD = 'CADNN_result/synthetic/training_results_sPOD/2022_08_23__19-55-17/net_weights/epoch_99.pt'
PATH_POD = 'CADNN_result/synthetic/training_results_sPOD/2022_08_23__19-55-17/net_weights/epoch_99.pt'

In [ ]:
# Training model for sPOD
train_model_sPOD = TrainingFramework(dict_network_sPOD, split=0.70, 
                                     log_folder='./CADNN_result/synthetic/training_results_sPOD/')
trained_model_sPOD = train_model_sPOD.training(epochs=100, save_every=50, print_every=50, 
                                               log_base_name='/', pretrained_weights=PATH_sPOD)

In [ ]:
import os
import pathlib

# Testing model for sPOD
testing_method = ''

log_folder_base = 'CADNN_result/synthetic/training_results_sPOD/'
log_folder_trained_model = sorted(pathlib.Path(log_folder_base).glob('*/'), key=os.path.getmtime)[-1]

test_model_sPOD = TestingFramework(dict_network_sPOD)
test_model_sPOD.testing(log_folder_trained_model=str(log_folder_trained_model), 
                        testing_method=testing_method, model=trained_model_sPOD)
results_predicted_sPOD = test_model_sPOD.time_amplitude_test_output

## -------------------------------------

In [ ]:
# Training model for POD
train_model_POD = TrainingFramework(dict_network_POD, split=0.80, 
                                    log_folder='./CADNN_result/synthetic/training_results_POD/')
trained_model_POD = train_model_POD.training(epochs=100, save_every=50, print_every=50, 
                                             log_base_name='/', pretrained_weights=PATH_POD)

In [ ]:
import os
import pathlib

# Testing model for POD
testing_method = ''

log_folder_base = 'CADNN_result/synthetic/training_results_POD/'
log_folder_trained_model = sorted(pathlib.Path(log_folder_base).glob('*/'), key=os.path.getmtime)[-1]

test_model_POD = TestingFramework(dict_network_POD)
test_model_POD.testing(log_folder_trained_model=str(log_folder_trained_model), 
                       testing_method=testing_method, model=trained_model_POD)
results_predicted_POD = test_model_POD.time_amplitude_test_output

## -------------------------------------

In [ ]:
# This cell is reserved for data manipulations for the online analysis
frame_amplitudes_predicted_sPOD = results_predicted_sPOD[:-2, :]
shifts_predicted_sPOD = results_predicted_sPOD[-2:, :]
frame_amplitudes_predicted_POD = results_predicted_POD

## Online error analysis

In [ ]:
df.onlineErroranalysis(frame_amplitudes_predicted_sPOD, shifts_predicted_sPOD, frame_amplitudes_predicted_POD)